In [ ]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

## Non-std libs
import matplotlib.pyplot as plt

## Local libs
from scn_rrd import rrd_utils

References
- https://github.com/librenms/librenms/blob/master/LibreNMS/Data/Source/FpingResponse.php :: `__construct()`
- https://github.com/librenms/librenms/blob/fb3c2ecb1c9275108058e76e8af6f145a92235e7/LibreNMS/Polling/ConnectivityHelper.php :: `savePingStats()`

In [ ]:
site_to_df = rrd_utils.collect_rrd_dataframes('ping', '-6month')

fig, ax = plt.subplots()
for (site, df) in site_to_df.items():
    ax.plot(df['time'], df['ping'], label=f'{site}')
ax.set_title('EPC latency')
ax.set_ylabel('millisec')
ax.legend()